<a href="https://colab.research.google.com/github/NISHTHA0305/ML_Model/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from google.colab import files
uploaded = files.upload()
df = pd.read_excel('Weather Data 2014-11-30.xlsx')
weather_data = pd.read_excel("Weather Data 2014-11-30.xlsx", sheet_name="Weather Data")

weather_data = weather_data.dropna(subset=["Date", "Time"]).reset_index(drop=True)

columns_to_fix = ["TempOut","OutHum","DewPt","WindSpeed","Bar","ArcInt"]
weather_data[columns_to_fix] = weather_data[columns_to_fix].replace('---', np.nan)

for col in columns_to_fix:
    weather_data[col] = pd.to_numeric(weather_data[col], errors='coerce')

weather_data.dropna(subset=columns_to_fix, inplace=True)

weather_data["Datetime"]=pd.to_datetime(weather_data["Date"].astype(str) + " "+ weather_data["Time"].astype(str))
weather_data["Hour"]=weather_data["Datetime"].dt.hour
weather_data["Day"]=weather_data["Datetime"].dt.day
weather_data["Month"]=weather_data["Datetime"].dt.month

scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(weather_data[["TempOut","OutHum","DewPt","WindSpeed","Bar","ArcInt"]])
window_size = 48
X = []
y = []
target_dates = weather_data.index[window_size:]

for i in range(window_size, len(scaled_data)):
    X.append(scaled_data[i - window_size:i, 0])
    y.append(scaled_data[i, 0])

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test, dates_train, dates_test = train_test_split(
    X, y, target_dates, test_size=0.2, shuffle=False
)

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
target_scaler = MinMaxScaler()
scaled_target = target_scaler.fit_transform(weather_data[['TempOut']].astype(float))

model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=128))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, y_train,epochs=100,batch_size=128, validation_split=0.1)

predictions = model.predict(X_test)
predictions = target_scaler.inverse_transform(predictions).flatten()
y_test = target_scaler.inverse_transform(y_test.reshape(-1,1)).flatten()

rmse = np.sqrt(np.mean((y_test - predictions)**2))
print(f'RMSE: {rmse:.2f}')

Saving Weather Data 2014-11-30.xlsx to Weather Data 2014-11-30 (2).xlsx


<ipython-input-3-a893a0413d29>:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  weather_data[columns_to_fix] = weather_data[columns_to_fix].replace('---', np.nan)


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


382/382 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0101 - val_loss: 8.1343e-04
Epoch 2/100
382/382 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.0012 - val_loss: 7.9207e-04
Epoch 3/100
382/382 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 8.7709e-04 - val_loss: 3.0377e-04
Epoch 4/100
382/382 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 7.6833e-04 - val_loss: 3.6573e-04
Epoch 5/100
382/382 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 6.4158e-04 - val_loss: 2.3051e-04
Epoch 6/100
382/382 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 5.5576e-04 - val_loss: 1.8990e-04
Epoch 7/100
382/382 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 5.0190e-04 - val_loss: 1.8531e-04
Epoch 8/100
382/382 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 4.6072e-04 - val_loss: 1.7691e-04
Epoch 9/100
382/382 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 4.0315e-04 - val_loss: 1.7076e-04
Epoch 10/100
382/382 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 3.7682e-04 - val_loss: 1.9032e-04
Epoch 11/100
382/382 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - lo